### A description of the problem and a discussion of the background. 

Tesla wants to open a new Tesla Store in order to sell more cars and therefore whats to find a new Location. Unfortunately, they lost all their data of their existing stores. So they turned to me in order to find a new location with the following requirements:

* The store has to be in a Capital of a European country
* There are no Tesla locations within 50km around the capital



### A description of the data and how it will be used to solve the problem

A list of European countries with the the capitals and locations can be found here: http://www.nobochamp.de/hauptstaedte2.html <br>
With BeautifulSoup the table is crawled, cleaned and stores in the Dataframe **real_real_df**<br>
With this available data a search with the foursquare API for the search term 'Tesla Store' is executed for all 50 European capitals within 50km radius in the table **real_real_df** <br>
The resulting count of results is written into the dataframe **df_final**

### Data cleaning

In [61]:
from bs4 import BeautifulSoup
import bs4
import requests
import pandas as pd
import numpy as np
import json
import folium

In [18]:
page = requests.get("http://www.nobochamp.de/hauptstaedte2.html")
if page.status_code == 200:
    content = page.content
table_contents=[]
soup = BeautifulSoup(content, 'html.parser')
table=soup.find('div')
data=table.findAll('div')

In [19]:
count=0
row=0
list=[]
dict={}
for data_point in data:    
    if count==4:        
        list.append(dict)
        dict={}
        count=0
    dict[count]=data_point.text    
    count+=1    
    
df=pd.DataFrame(list)#
df_real=df.loc[0:df[df[0]=='Svalbard (NO)'].index[0]]

df_real.columns=['European States', 'European Capitals','Coordinates (φ / λ)','Altitude in meters above sea level']

In [20]:
coordinates=df_real['Coordinates (φ / λ)']

list_2=[]
for element in coordinates:
    dict_2={}
    dict_2['φ'],v=element.split('/')
    dict_2['λ']=v.strip()
    list_2.append(dict_2)
coord_df=pd.DataFrame(list_2)

In [21]:
capitals=df_real['European Capitals']
list_3=[]
for element in capitals:
    dict_3={}
    if '/' in element:
        _,value=element.split('/')
    else:
        value=element
    dict_3['Capital']=value
    list_3.append(dict_3)
    
capitals_df=pd.DataFrame(list_3)


In [22]:
real_real_df=pd.concat([df_real,capitals_df,coord_df],axis=1)
real_real_df.drop(0,inplace=True)#data cleaning
real_real_df.drop('Coordinates (φ / λ)',inplace=True,axis=1)#data cleaning
real_real_df.drop('European Capitals',inplace=True,axis=1)#data cleaning
real_real_df.drop('Altitude in meters above sea level',inplace=True,axis=1)#data cleaning
real_real_df

,European States,Capital,φ,λ
1,Albania,Tirana,+41.3317,+19.8172
2,Andorra,Andorra la Vella,+42.5075,+1.5218
3,Austria,Vienna,+48.2092,+16.3728
4,Belarus,Minsk,+53.9678,+27.5766
5,Belgium,Brussels,+50.8371,+4.3676
6,Bosnia and Herzegovina,Sarajevo,+43.8608,+18.4214
7,Bulgaria,Sofia,+42.7105,+23.3238
8,Croatia,Zagreb,+45.8150,+15.9785
9,Czech Republic,Prague,+50.0878,+14.4205
10,Denmark,Copenhagen,+55.6763,+12.5681


### Foursquare API

In [23]:
CLIENT_ID = 'T1BFYE1IJPCDMCIU3T5XYT2Z3PULJHUQOEDMWX2RWWKK4JRJ' # your Foursquare ID
CLIENT_SECRET = 'MNE0GRC2BKSDRJXPQVI4ATMH3W5NIDWXX1GTZYYOUIF4G4BD' # your Foursquare Secret
ACCESS_TOKEN = 'P4LWXYDTPEPSC4Y5HWDYSPMNCYYGSCDZ2TXXK0LSKLM1AMI2' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 10 #retricted
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T1BFYE1IJPCDMCIU3T5XYT2Z3PULJHUQOEDMWX2RWWKK4JRJ
CLIENT_SECRET:MNE0GRC2BKSDRJXPQVI4ATMH3W5NIDWXX1GTZYYOUIF4G4BD


In [24]:
search_query = 'Tesla Store'
radius = 50000
print(search_query + ' .... OK!')

Tesla Store .... OK!


In [25]:
i=14
latitude=real_real_df['φ'][i]
longitude=real_real_df['λ'][i]
print(latitude)
print(longitude)

+52.5235 
+13.4115


In [85]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=T1BFYE1IJPCDMCIU3T5XYT2Z3PULJHUQOEDMWX2RWWKK4JRJ&client_secret=MNE0GRC2BKSDRJXPQVI4ATMH3W5NIDWXX1GTZYYOUIF4G4BD&ll=62.0177,-6.7719&oauth_token=P4LWXYDTPEPSC4Y5HWDYSPMNCYYGSCDZ2TXXK0LSKLM1AMI2&v=20180604&query=Tesla Store&radius=50000&limit=10'

In [27]:
results = requests.get(url).json()


In [28]:
dict_cvc=json.loads(json.dumps(results))

In [29]:
len(dict_cvc['response']['venues'])

10

In [48]:
dict_final={}
list_final=[]
for i in real_real_df.index:
    dict_final={}
    dict_final['country']=real_real_df['European States'][i]
    dict_final['city']=real_real_df['Capital'][i]
    dict_final['latitude']=real_real_df['φ'][i]
    dict_final['longitude']=real_real_df['λ'][i]
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dict_final['latitude'], dict_final['longitude'],ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

    results = requests.get(url).json()
    
    dict_cvc=json.loads(json.dumps(results))
    dict_final['count_Tesla_results (max 10)']=len(dict_cvc['response']['venues'])
    
    list_final.append(dict_final)
df_final=pd.DataFrame(list_final)
    

In [51]:
df_final.sort_values('count_Tesla_results (max 10)',inplace=True)

In [55]:
df_final.reset_index(inplace=True)

In [56]:
df_final

,index,country,city,latitude,longitude,count_Tesla_results (max 10)
0,44,Faroe Islands (DK),Torshavn (on Streymoy),+62.0177,-6.7719,0
1,49,Svalbard (NO),Longyearbyen,+78.2186,+15.6488,3
2,1,Andorra,Andorra la Vella,+42.5075,+1.5218,6
3,27,Montenegro,Podgorica,+42.4602,+19.2595,10
4,28,Netherlands,Amsterdam,+52.3738,+4.8910,10
5,29,North Macedonia,Skopje,+42.0024,+21.4361,10
6,30,Norway,Oslo,+59.9138,+10.7387,10
7,31,Poland,Warsaw,+52.2297,+21.0122,10
8,32,Portugal,Lisbon,+38.7072,-9.1355,10
9,33,Romania,Bucharest,+44.4479,+26.0979,10


### Conclusion

Result is clear Tesla have to open a  new store in Torshavn on Faroe Islands!

In [84]:
latitude=float(df_final['latitude'].loc[0])
longitude=float(df_final['longitude'].loc[0])

country=df_final['country'].loc[0]
city=df_final['city'].loc[0]
count=df_final['count_Tesla_results (max 10)'].loc[0]

# create map of New York using latitude and longitude values
map_result = folium.Map(location=[latitude, longitude], zoom_start=5)


label = '{}, {}:{}'.format(country, city,count)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude, longitude],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_result)  
    
map_result

In [63]:
latitude=df_final['latitude'].loc[0]
longitude=df_final['longitude'].loc[0]

In [65]:
float(latitude)

62.0177

In [59]:
longitude

'-6.7719'